In [103]:
import pandas as pd
import numpy as np
from scipy import stats
import requests
import matplotlib.pyplot as plt
import seaborn as sns
from statsmodels.distributions.empirical_distribution import ECDF
from functools import partial

# Data Handling
This notebook slices the data into training data and testing data

## Data
One single dataset from Bloomberg. 
- Daily closing price of BTC future and Daily Closing value of brr. 
- Data is merged by Bloomberg using the compare function


In [104]:
future_brr = pd.read_csv('../data/cleaned_data/btc_future_brr_new.csv')

In [105]:
future_brr

,Unnamed: 0,Date,btc future last px,brr last px,log return future,log return bitcoin
0,1,2021-05-20,40320.0,41861.09,0.024606,0.158251
1,2,2021-05-19,39340.0,35734.10,-0.092441,-0.187849
2,3,2021-05-18,43150.0,43118.61,-0.020870,-0.028939
3,4,2021-05-17,44060.0,44384.66,-0.130263,-0.134395
4,8,2021-05-14,50190.0,50769.13,0.037039,0.003599
...,...,...,...,...,...,...
822,1345,2018-02-15,10090.0,9842.22,0.082606,0.061949
823,1346,2018-02-14,9290.0,9251.01,0.065616,0.080712
824,1347,2018-02-13,8700.0,8533.68,-0.018224,-0.008991
825,1348,2018-02-12,8860.0,8610.75,0.028622,0.028990


In [ ]:
plt.plot()

In [107]:
future_brr = future_brr.iloc[:, 1:]

## Slicing data

In [94]:
train_size = 300
test_size = 100
step_size = 5

In [95]:
len(future_brr)

827

In [96]:
data_name = 'future_brr_new_v5'

In [97]:
# n = int(np.floor((len(future_brr)-train_size)/step_size))
n = int(np.floor((len(future_brr) - train_size - test_size)/step_size))
train_set = []
test_set = []

for i in range(n):
    data = future_brr.iloc[(i*step_size):((i)*step_size)+train_size+test_size]
    test = data.iloc[:test_size,]
    train = data.iloc[test_size:,]
    train_set.append(train)
    test_set.append(test)

for i in range(len(train_set)):
    train_set[i].to_csv("../processed_data/%s/train/" %data_name +str(i)+".csv")
    test_set[i].to_csv("../processed_data/%s/test/" %data_name +str(i)+".csv")

In [98]:
test_set[0].head()

,Date,btc future last px,brr last px,log return future,log return bitcoin
0,2021-05-20,40320.0,41861.09,0.024606,0.158251
1,2021-05-19,39340.0,35734.10,-0.092441,-0.187849
2,2021-05-18,43150.0,43118.61,-0.020870,-0.028939
3,2021-05-17,44060.0,44384.66,-0.130263,-0.134395
4,2021-05-14,50190.0,50769.13,0.037039,0.003599


In [99]:
train_set[0].head()

,Date,btc future last px,brr last px,log return future,log return bitcoin
100,2020-12-28,27110.0,27004.91,0.155880,0.152007
101,2020-12-24,23197.0,23196.73,-0.013403,-0.019712
102,2020-12-23,23510.0,23658.53,0.002769,0.006825
103,2020-12-22,23445.0,23497.61,0.024394,0.018023
104,2020-12-21,22880.0,23077.91,0.000875,0.025738


In [100]:
train_set[0].tail()

,Date,btc future last px,brr last px,log return future,log return bitcoin
395,2019-10-25,7678.0,7678.49,0.024790,0.025366
396,2019-10-24,7490.0,7486.17,0.002005,0.001125
397,2019-10-23,7475.0,7477.75,-0.088293,-0.095183
398,2019-10-22,8165.0,8224.48,-0.004887,-0.002658
399,2019-10-21,8205.0,8246.37,0.030315,0.040525


In [101]:
test_set[1].head()

,Date,btc future last px,brr last px,log return future,log return bitcoin
5,2021-05-13,48365.0,50586.73,-0.118415,-0.113057
6,2021-05-12,54445.0,56641.72,-0.042433,0.015200
7,2021-05-11,56805.0,55787.27,0.019554,-0.028760
8,2021-05-10,55705.0,57415.03,-0.036227,0.005824
9,2021-05-07,57760.0,57081.64,0.027736,-0.003132


In [102]:
train_set[-1].tail()

,Date,btc future last px,brr last px,log return future,log return bitcoin
815,2018-02-27,10660.0,10553.75,0.039221,0.033284
816,2018-02-26,10250.0,10208.26,-0.006709,-0.010831
817,2018-02-23,10319.0,10319.43,0.022938,0.030236
818,2018-02-22,10085.0,10012.08,-0.027868,-0.054050
819,2018-02-21,10370.0,10568.12,-0.123659,-0.080984
